### Импорты библиотек

In [95]:
import pandas as pd
import numpy as np

import natasha

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,

    Doc
)

from yargy.tokenizer import MorphTokenizer
from ipymarkup import show_line_markup

from yargy import rule, Parser, or_
from yargy.predicates import normalized
from yargy.pipelines import morph_pipeline

### Загрузка данных

In [2]:
df = pd.read_csv(r"D:\code\Projects_for_resume\test_nlp\test_data_copy.csv")

In [3]:
df

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


In [4]:
df_mng = df.drop(df[df.role == 'client'].index)

In [5]:
df_mng

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет
...,...,...,...,...
472,5,135,manager,Так дмитрий
473,5,136,manager,Все записала тогда завтра ждите звонка
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...


### Приветствие

In [6]:
df_mng['insight'] = np.nan

In [7]:
def greetings(line):
    """Функция осуществляет поиск реплик, в которых менеджер поздоровался с клиентом"""
    
    VARIANTS = morph_pipeline([
        'здравствуйте',
        'добрый день'
    ])

    GREETINGS = rule(VARIANTS)
    parser = Parser(GREETINGS)
    matches = list(parser.findall(line))
    flag = 1 if len(matches) != 0 else 0
    return f'greeting=True' if flag else f'greeting=False'

In [8]:
df_mng['insight'] = df_mng['text'].apply(greetings)
df_mng

,dlg_id,line_n,role,text,insight
1,0,1,manager,Алло здравствуйте,greeting=True
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,greeting=False
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,greeting=False
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,greeting=False
11,0,11,manager,Что для вас приоритет,greeting=False
...,...,...,...,...,...
472,5,135,manager,Так дмитрий,greeting=False
473,5,136,manager,Все записала тогда завтра ждите звонка,greeting=False
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,greeting=False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,greeting=False


### Представление себя клиенту

In [9]:
def introduce_youself(line):
    """Функция осуществляет поиск реплик, в которых менеджер представился 
    в начале диалога с клиентом"""
    
    VARIANTS = morph_pipeline([
        'меня зовут',
        'зовут'
    ])

    INTROS = rule(VARIANTS)
    parser = Parser(INTROS)
    matches = list(parser.findall(line))
    flag = 1 if len(matches) != 0 else 0
    return 'introducing=True' if flag else 'introducing=False'

In [10]:
df_mng['insight'] = df_mng['text'].apply(introduce_youself)
df_mng

,dlg_id,line_n,role,text,insight
1,0,1,manager,Алло здравствуйте,introducing=False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,introducing=True
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,introducing=False
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,introducing=False
11,0,11,manager,Что для вас приоритет,introducing=False
...,...,...,...,...,...
472,5,135,manager,Так дмитрий,introducing=False
473,5,136,manager,Все записала тогда завтра ждите звонка,introducing=False
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,introducing=False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,introducing=False


### Имя

In [ ]:
# В с случае с извлечением имени возникли сложности из-за капитализации. 
# Код в ячейках ниже отрабатывает при поиске имен, прописанных с заглавной буквы.
# К сожалению, я не успел адаптировать его для учета случаев, когда имена начинаются с маленькой буквы.

In [104]:
text = '''Алло здравствуйте
Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления 
лицензии а мы с серым у вас скоро срок заканчивается
Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика 
сравнивать
Угу а на что вы обращаете внимание при выборе
Что для вас приоритет
Ну у вас срок заканчивается поэтому мы набрали + ко всему
А так нет не только поэтому просто я обратила внимание и вы пользуетесь 
виджетами мы там с вами сотрудничали
А если вы 19 являетесь то лучше то идти бесплатно
Ага хорошо
Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то 
например не с кем сейчас справляться кто то сравнивал
Да мы услышали вас спасибо за рекомендации
Вот понимаю да но дело в том что в основном да за месяц
Необходимо обзванивать
Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду
И вам спасибо большое за обратную связь
Всего хорошего до свидания'''

In [105]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [106]:
doc = Doc(text)
doc.segment(segmenter) 
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [107]:
doc.segment(segmenter)

In [108]:
doc.tag_morph(morph_tagger)

In [109]:
for span in doc.spans:
    span.normalize(morph_vocab)

{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{}

In [110]:
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)

names_dict = {_.normal: _.fact.as_dict for _ in doc.spans if _.fact}
names_dict

{'Ангелина': {'first': 'Ангелина'}}

In [111]:
list(names_dict.keys())

['Ангелина']

### Название компании

In [13]:
def company_name(line):
    """Функция осуществляет поиск реплик, в которых менеджер упомянул 
    название компании, которую он представляет."""
    
    VARIANTS = morph_pipeline([
    'компания'
    ])

    COMPANIES = rule(VARIANTS)
    parser = Parser(COMPANIES)
    matches = list(parser.findall(line))
    flag = 1 if len(matches) != 0 else 0
    return 'company_name=True' if flag else 'company_name=False'

In [14]:
df_mng['insight'] = df_mng['text'].apply(company_name)
df_mng

,dlg_id,line_n,role,text,insight
1,0,1,manager,Алло здравствуйте,company_name=False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,company_name=True
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,company_name=False
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,company_name=False
11,0,11,manager,Что для вас приоритет,company_name=False
...,...,...,...,...,...
472,5,135,manager,Так дмитрий,company_name=False
473,5,136,manager,Все записала тогда завтра ждите звонка,company_name=False
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,company_name=False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,company_name=False


### Прощание

In [15]:
def farewell(line):
    """Функция извлекает реплики, в которых менеджер 
    прощается с клиентом."""
    
    VARIANTS = morph_pipeline([
         'до свидания',
         'всего доброго'
     ])

    FAREWELLS = rule(VARIANTS)
    parser = Parser(FAREWELLS)
    matches = list(parser.findall(line))
    flag = 1 if len(matches) != 0 else 0
    return 'farewell=True' if flag else 'farewell=False'

In [16]:
df_mng['insight'] = df_mng['text'].apply(farewell)
df_mng

,dlg_id,line_n,role,text,insight
1,0,1,manager,Алло здравствуйте,farewell=False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,farewell=False
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,farewell=False
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,farewell=False
11,0,11,manager,Что для вас приоритет,farewell=False
...,...,...,...,...,...
472,5,135,manager,Так дмитрий,farewell=False
473,5,136,manager,Все записала тогда завтра ждите звонка,farewell=False
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,farewell=False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,farewell=False


In [17]:
df_mng[df_mng.insight == 'farewell=True']

,dlg_id,line_n,role,text,insight
108,0,108,manager,Всего хорошего до свидания,farewell=True
163,1,54,manager,До свидания,farewell=True
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,farewell=True
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,farewell=True
479,5,142,manager,Ну до свидания хорошего вечера,farewell=True


### Приветствие и прощание в каждом диалоге

In [94]:
dialogs = []
for i in range(0, 6):
    d = ' '.join(df_mng.text[df_mng.dlg_id == i])
    dialogs.append(d)
print(dialogs[0])

Алло здравствуйте Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать Угу а на что вы обращаете внимание при выборе Что для вас приоритет Ну у вас срок заканчивается поэтому мы набрали + ко всему А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали А если вы 19 являетесь то лучше то идти бесплатно Ага хорошо Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал Да мы услышали вас спасибо за рекомендации Вот понимаю да но дело в том что в основном да за месяц Необходимо обзванивать Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду И вам спасибо большое за обратную связь Всего хорошего до свидания


In [90]:
def greetings_farewells(line):
    """Функция осуществляет поиск реплик, в которых менеджер поздоровался и попрощался с клиентом"""
    
    VARIANTS = morph_pipeline([
        'здравствуйте',
        'добрый день'
    ])

    GREETINGS = rule(VARIANTS)
    parser = Parser(GREETINGS)
    matches = list(parser.findall(line))
    flag = 1 if len(matches) != 0 else 0
    
    VARIANTS_2 = morph_pipeline([
         'до свидания',
         'всего доброго'
     ])

    FAREWELLS = rule(VARIANTS_2)
    parser = Parser(FAREWELLS)
    matches_2 = list(parser.findall(line))
    
    flag = 1 if (len(matches) != 0) and (len(matches_2) != 0)  else 0
    
    return f'Менеджер выполнил требование по общению с клиентом' if flag else f'Менеджер НЕ выполнил требование по общению с клиентом'

In [91]:
for d in dialogs:
    print(greetings_farewells(d))

Менеджер выполнил требование по общению с клиентом
Менеджер выполнил требование по общению с клиентом
Менеджер НЕ выполнил требование по общению с клиентом
Менеджер выполнил требование по общению с клиентом
Менеджер НЕ выполнил требование по общению с клиентом
Менеджер НЕ выполнил требование по общению с клиентом
